# Model Training and Evaluation

This notebook covers the definition, training, and evaluation of a transformer-based multi-label text classification model.

## 1. Load Prepared Datasets and Libraries

Load the PyTorch datasets and import required libraries for model training and evaluation.

In [ ]:
# Import libraries and load datasets
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
import numpy as np

In [7]:
# Define ReviewsDataset class (must match the one used in 03_modeling.ipynb)
from torch.utils.data import Dataset
class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.float32)
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

In [8]:
train_data = torch.load('../data/processed/train_dataset.pt', weights_only=False)
test_data = torch.load('../data/processed/test_dataset.pt', weights_only=False)

# Reuse your ReviewsDataset class definition here
train_dataset = ReviewsDataset(train_data['encodings'], train_data['labels'])
test_dataset = ReviewsDataset(test_data['encodings'], test_data['labels'])

## 2. Model Definition and Configuration

Define the transformer model for multi-label classification and set up optimizer, loss, and training parameters.

In [10]:
# Model definition and configuration
num_labels = train_data['labels'].shape[1]
print(f"Number of labels: {num_labels}")

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels,
    problem_type='multi_label_classification'
)

# Training parameters
batch_size = 16
epochs = 3
learning_rate = 2e-5

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Loss function for multi-label
loss_fn = torch.nn.BCEWithLogitsLoss()

Number of labels: 4


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Model Training and Evaluation

Train the model and evaluate its performance on the test set.

In [11]:
# Training and evaluation loop (template)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs} - Training loss: {avg_train_loss:.4f}")

# Evaluation (template)
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits.cpu().numpy()
        preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
        all_preds.append(preds)
        all_labels.append(labels)

import numpy as np
all_preds = np.vstack(all_preds)
all_labels = np.vstack(all_labels)

# Compute metrics 
from sklearn.metrics import accuracy_score, f1_score
print("Test accuracy:", accuracy_score(all_labels, all_preds))
print("Test macro F1:", f1_score(all_labels, all_preds, average='macro'))

KeyboardInterrupt: 